I will use several data sources to prepare the final dataset.

First, I will take data on incidents involving cyclists in London from the
https://bikedata.cyclestreets.net/#11.84/51.49989/-0.06585

Here I select the London location and select Cyclists as participants in the incidents. 

Such a query gave me a dataset with 138 incidents for years 2000 - 2020, which I download in csv format. 

The dataset has data on:

Timestamp
Severity
Number of Casualties 
Number of Vehicles 
Latitude 
Longitude

I'm interested in trying Tableau tool for further visualisations, so incident coordinates are what I need.

Also, I want to see what the weather was like on the days of the incidents. 
My large dataset for bicycle incidents in UK contains weather and road conditions. 

I'll try to get similar information for my London dataset.

After exploring ways to take historical weather data from various APIs, this time I settled on a quick solution - I found a library that can pull and agrigate historical weather data from various sources.

https://github.com/meteostat/meteostat-python

I got a file with historical data from 2000 to 2020. 
I'm going to filter and transform the data to find weather conditions on the days of the London incidents to get more insights.


In [1]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations


In [206]:
import pandas as pd

In [4]:

# Set time period
start = datetime(2000, 3, 21)
end = datetime(2020, 12, 8)


In [5]:

# Create Point for London UK
London = Point(51.509865, -0.118092, 70)


In [6]:
# Get daily data for period
data = Daily(London, start, end)
data = data.fetch()
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7568 entries, 2000-03-21 to 2020-12-08
Freq: D
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tavg    7568 non-null   float64
 1   tmin    7480 non-null   float64
 2   tmax    7506 non-null   float64
 3   prcp    6648 non-null   float64
 4   snow    92 non-null     float64
 5   wdir    7148 non-null   float64
 6   wspd    7366 non-null   float64
 7   wpgt    801 non-null    float64
 8   pres    7150 non-null   float64
 9   tsun    0 non-null      float64
dtypes: float64(10)
memory usage: 650.4 KB


Column	Description	Type
station	The Meteostat ID of the weather station (only if query refers to multiple stations)	String
time	The date	Datetime64
tavg	The average air temperature in °C	Float64
tmin	The minimum air temperature in °C	Float64
tmax	The maximum air temperature in °C	Float64
prcp	The daily precipitation total in mm	Float64
snow	The snow depth in mm	Float64
wdir	The average wind direction in degrees (°)	Float64
wspd	The average wind speed in km/h	Float64
wpgt	The peak wind gust in km/h	Float64
pres	The average sea-level air pressure in hPa	Float64
tsun	The daily sunshine total in minutes (m)	Float64

In [37]:
#save csv
data.to_csv('/Users/kristina/Documents/Velo_python/weather_london_data.csv')

In [44]:
#save all in dataframe
df_w = pd.read_csv('/Users/kristina/Documents/Velo_python/weather_london_data.csv')

In [45]:
#check 

df_w.info()
df_w.count()
df_w

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7568 entries, 0 to 7567
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    7568 non-null   object 
 1   tavg    7568 non-null   float64
 2   tmin    7480 non-null   float64
 3   tmax    7506 non-null   float64
 4   prcp    6648 non-null   float64
 5   snow    92 non-null     float64
 6   wdir    7148 non-null   float64
 7   wspd    7366 non-null   float64
 8   wpgt    801 non-null    float64
 9   pres    7150 non-null   float64
 10  tsun    0 non-null      float64
dtypes: float64(10), object(1)
memory usage: 650.5+ KB


,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2000-03-21,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
1,2000-03-22,8.3,2.8,15.8,NaN,NaN,NaN,1.0,NaN,1016.6,NaN
2,2000-03-23,9.8,4.3,14.4,NaN,NaN,NaN,11.1,NaN,1010.6,NaN
3,2000-03-24,10.1,6.9,14.2,2.8,NaN,206.0,12.7,NaN,1006.5,NaN
4,2000-03-25,8.3,5.5,11.5,3.0,NaN,215.0,13.9,NaN,1009.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7563,2020-12-04,3.1,2.0,3.8,15.0,NaN,260.0,17.3,35.2,977.3,NaN
7564,2020-12-05,3.2,1.4,5.0,0.0,NaN,106.0,13.8,27.8,991.5,NaN
7565,2020-12-06,1.7,0.9,3.3,0.0,NaN,13.0,11.0,25.9,1001.8,NaN
7566,2020-12-07,0.8,-0.2,2.1,0.3,NaN,25.0,6.5,16.7,999.3,NaN


In [46]:
#пrename Time to Date
#convert Date into datetime 

df_w.rename(columns={'time':'Date'}, inplace=True)
df_w['Date'] = pd.to_datetime(df_w['Date'])
df_w

,Date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2000-03-21,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
1,2000-03-22,8.3,2.8,15.8,NaN,NaN,NaN,1.0,NaN,1016.6,NaN
2,2000-03-23,9.8,4.3,14.4,NaN,NaN,NaN,11.1,NaN,1010.6,NaN
3,2000-03-24,10.1,6.9,14.2,2.8,NaN,206.0,12.7,NaN,1006.5,NaN
4,2000-03-25,8.3,5.5,11.5,3.0,NaN,215.0,13.9,NaN,1009.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7563,2020-12-04,3.1,2.0,3.8,15.0,NaN,260.0,17.3,35.2,977.3,NaN
7564,2020-12-05,3.2,1.4,5.0,0.0,NaN,106.0,13.8,27.8,991.5,NaN
7565,2020-12-06,1.7,0.9,3.3,0.0,NaN,13.0,11.0,25.9,1001.8,NaN
7566,2020-12-07,0.8,-0.2,2.1,0.3,NaN,25.0,6.5,16.7,999.3,NaN


In [47]:
#set date as index

df_w.set_index('Date', inplace=True)
df_w.info()
df_w

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7568 entries, 2000-03-21 to 2020-12-08
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tavg    7568 non-null   float64
 1   tmin    7480 non-null   float64
 2   tmax    7506 non-null   float64
 3   prcp    6648 non-null   float64
 4   snow    92 non-null     float64
 5   wdir    7148 non-null   float64
 6   wspd    7366 non-null   float64
 7   wpgt    801 non-null    float64
 8   pres    7150 non-null   float64
 9   tsun    0 non-null      float64
dtypes: float64(10)
memory usage: 650.4 KB


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
Date,,,,,,,,,,
2000-03-21,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
2000-03-22,8.3,2.8,15.8,NaN,NaN,NaN,1.0,NaN,1016.6,NaN
2000-03-23,9.8,4.3,14.4,NaN,NaN,NaN,11.1,NaN,1010.6,NaN
2000-03-24,10.1,6.9,14.2,2.8,NaN,206.0,12.7,NaN,1006.5,NaN
2000-03-25,8.3,5.5,11.5,3.0,NaN,215.0,13.9,NaN,1009.3,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-12-04,3.1,2.0,3.8,15.0,NaN,260.0,17.3,35.2,977.3,NaN
2020-12-05,3.2,1.4,5.0,0.0,NaN,106.0,13.8,27.8,991.5,NaN
2020-12-06,1.7,0.9,3.3,0.0,NaN,13.0,11.0,25.9,1001.8,NaN


In [48]:
df_w

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
Date,,,,,,,,,,
2000-03-21,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
2000-03-22,8.3,2.8,15.8,NaN,NaN,NaN,1.0,NaN,1016.6,NaN
2000-03-23,9.8,4.3,14.4,NaN,NaN,NaN,11.1,NaN,1010.6,NaN
2000-03-24,10.1,6.9,14.2,2.8,NaN,206.0,12.7,NaN,1006.5,NaN
2000-03-25,8.3,5.5,11.5,3.0,NaN,215.0,13.9,NaN,1009.3,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-12-04,3.1,2.0,3.8,15.0,NaN,260.0,17.3,35.2,977.3,NaN
2020-12-05,3.2,1.4,5.0,0.0,NaN,106.0,13.8,27.8,991.5,NaN
2020-12-06,1.7,0.9,3.3,0.0,NaN,13.0,11.0,25.9,1001.8,NaN


In [203]:
#take a file with London accidents with coordinates

w = pd.read_csv('/Users/kristina/Documents/Velo_python/collisions_London.csv')
w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Accident Index        138 non-null    object 
 1   Timestamp             138 non-null    object 
 2   severity              138 non-null    object 
 3   Casualties            138 non-null    object 
 4   Number of Casualties  138 non-null    int64  
 5   Number of Vehicles    138 non-null    int64  
 6   url                   138 non-null    object 
 7   apiUrl                138 non-null    object 
 8   Latitude              138 non-null    float64
 9   Longitude             138 non-null    float64
dtypes: float64(2), int64(2), object(6)
memory usage: 10.9+ KB


In [204]:
#convert date in datetime and take only date (without time)


w['Date']= pd.to_datetime(w['Timestamp']).dt.date

In [68]:
#set date as Index

w.set_index('Date', inplace=True)

In [69]:
#sort dataframe on index

w = w.sort_values(by = 'Date')
df_w = df_w.sort_values(by = 'Date')

In [71]:
#merge two dataframes on dates

all_weather = w.merge(df_w, left_index = True, right_index = True)
all_weather

,Accident Index,Timestamp,severity,Casualties,Number of Casualties,Number of Vehicles,url,apiUrl,Latitude,Longitude,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
Date,,,,,,,,,,,,,,,,,,,,
2000-03-21,200001CP00079,"8:06am, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510986,-0.102536,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
2000-03-21,200001CP00131,"6:00pm, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511196,-0.104401,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
2000-07-27,200001CP00264,"5:58pm, 27th July 2000",serious,Cyclist,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511908,-0.103939,19.3,16.0,23.8,NaN,NaN,228.0,10.0,NaN,1012.5,NaN
2000-08-09,200001CP00309,"1:00pm, 9th August 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510995,-0.103112,19.1,16.0,22.7,NaN,NaN,200.0,11.6,NaN,1018.6,NaN
2000-11-20,200001CP00438,"6:15am, 20th November 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511470,-0.104678,5.8,4.0,6.9,NaN,NaN,221.0,9.5,NaN,995.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-06,2019480951155,"6:30pm, 6th December 2019",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511124,-0.107082,9.2,7.1,10.8,1.5,NaN,245.0,25.2,46.3,1006.0,NaN
2019-12-30,2019480932151,"3:28pm, 30th December 2019",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511032,-0.103036,5.8,3.7,9.5,0.0,NaN,223.0,11.7,25.9,1029.4,NaN
2020-09-28,2020480990431,"12:10pm, 28th September 2020",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.512006,-0.104307,11.7,8.4,15.8,0.0,NaN,299.0,13.7,29.6,1013.4,NaN


In [25]:
#rename columns

all_weather.rename(columns={'tmax':'Max temperature C',
                            'prcp':'Daily precipitation mm',
                            'snow':'Snow depth mm',
                            'wspd':'Wind speed km/h',
                            'wpgt': 'Peak wind gust in km/h'
                           }, inplace=True)

In [26]:
all_weather

,Timestamp,Month,Day,index,Max temperature C,Daily precipitation mm,Snow depth mm,Wind speed km/h,Peak wind gust in km/h
Date,,,,,,,,,
2000-03-21,2000-03-21 08:06:00,March,Tuesday,0,13.7,NaN,NaN,10.2,NaN
2000-03-21,2000-03-21 18:00:00,March,Tuesday,0,13.7,NaN,NaN,10.2,NaN
2000-07-27,2000-07-27 17:58:00,July,Thursday,128,23.8,NaN,NaN,10.0,NaN
2000-08-09,2000-08-09 13:00:00,August,Wednesday,141,22.7,NaN,NaN,11.6,NaN
2000-11-20,2000-11-20 06:15:00,November,Monday,244,6.9,NaN,NaN,9.5,NaN
...,...,...,...,...,...,...,...,...,...
2019-12-06,2019-12-06 18:30:00,December,Friday,7199,10.8,1.5,NaN,25.2,46.3
2019-12-30,2019-12-30 15:28:00,December,Monday,7223,9.5,0.0,NaN,11.7,25.9
2020-09-28,2020-09-28 12:10:00,September,Monday,7496,15.8,0.0,NaN,13.7,29.6


Weather conditions I need to get: Rain, Snow, Fog, Clear, Clear and windy, Rain and windy, Snow and windy

In [72]:
#I put everything into csv

all_weather.to_csv('/Users/kristina/Documents/Velo_python/allweather_data.csv')

In [106]:
df = pd.read_csv('/Users/kristina/Documents/Velo_python/allweather_data.csv')

In [107]:
df

,Date,Accident Index,Timestamp,severity,Casualties,Number of Casualties,Number of Vehicles,url,apiUrl,Latitude,...,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2000-03-21,200001CP00079,"8:06am, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510986,...,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
1,2000-03-21,200001CP00131,"6:00pm, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511196,...,8.3,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN
2,2000-07-27,200001CP00264,"5:58pm, 27th July 2000",serious,Cyclist,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511908,...,19.3,16.0,23.8,NaN,NaN,228.0,10.0,NaN,1012.5,NaN
3,2000-08-09,200001CP00309,"1:00pm, 9th August 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510995,...,19.1,16.0,22.7,NaN,NaN,200.0,11.6,NaN,1018.6,NaN
4,2000-11-20,200001CP00438,"6:15am, 20th November 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511470,...,5.8,4.0,6.9,NaN,NaN,221.0,9.5,NaN,995.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,2019480951155,"6:30pm, 6th December 2019",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511124,...,9.2,7.1,10.8,1.5,NaN,245.0,25.2,46.3,1006.0,NaN
134,2019-12-30,2019480932151,"3:28pm, 30th December 2019",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511032,...,5.8,3.7,9.5,0.0,NaN,223.0,11.7,25.9,1029.4,NaN
135,2020-09-28,2020480990431,"12:10pm, 28th September 2020",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.512006,...,11.7,8.4,15.8,0.0,NaN,299.0,13.7,29.6,1013.4,NaN
136,2020-10-02,2020480992250,"7:16am, 2nd October 2020",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511075,...,11.1,9.9,11.8,3.6,NaN,68.0,19.8,40.8,988.7,NaN


In [108]:
#create an empty column for weather conditions


df["Weather_conditions"] = ""

In [109]:
#I rename columns with dictionary

df.rename(columns={'tmax':'Max temperature C',
                   'severity':'Severity',
                   'tmin':'Min temperature C',
                            'prcp':'Daily precipitation mm',
                            'snow':'Snow depth mm',
                            'wspd':'Wind speed kmh',
                            'wpgt': 'Peak wind gust in kmh'
                           }, inplace=True)

In [110]:
df

,Date,Accident Index,Timestamp,Severity,Casualties,Number of Casualties,Number of Vehicles,url,apiUrl,Latitude,...,Min temperature C,Max temperature C,Daily precipitation mm,Snow depth mm,wdir,Wind speed kmh,Peak wind gust in kmh,pres,tsun,Weather_conditions
0,2000-03-21,200001CP00079,"8:06am, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510986,...,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN,
1,2000-03-21,200001CP00131,"6:00pm, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511196,...,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN,
2,2000-07-27,200001CP00264,"5:58pm, 27th July 2000",serious,Cyclist,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511908,...,16.0,23.8,NaN,NaN,228.0,10.0,NaN,1012.5,NaN,
3,2000-08-09,200001CP00309,"1:00pm, 9th August 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510995,...,16.0,22.7,NaN,NaN,200.0,11.6,NaN,1018.6,NaN,
4,2000-11-20,200001CP00438,"6:15am, 20th November 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511470,...,4.0,6.9,NaN,NaN,221.0,9.5,NaN,995.6,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,2019480951155,"6:30pm, 6th December 2019",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511124,...,7.1,10.8,1.5,NaN,245.0,25.2,46.3,1006.0,NaN,
134,2019-12-30,2019480932151,"3:28pm, 30th December 2019",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511032,...,3.7,9.5,0.0,NaN,223.0,11.7,25.9,1029.4,NaN,
135,2020-09-28,2020480990431,"12:10pm, 28th September 2020",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.512006,...,8.4,15.8,0.0,NaN,299.0,13.7,29.6,1013.4,NaN,
136,2020-10-02,2020480992250,"7:16am, 2nd October 2020",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511075,...,9.9,11.8,3.6,NaN,68.0,19.8,40.8,988.7,NaN,


In [111]:
#rename all columns with _

df = df.rename(columns=lambda col: col.replace(" ", "_"))

In [112]:
df

,Date,Accident_Index,Timestamp,Severity,Casualties,Number_of_Casualties,Number_of_Vehicles,url,apiUrl,Latitude,...,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Snow_depth_mm,wdir,Wind_speed_kmh,Peak_wind_gust_in_kmh,pres,tsun,Weather_conditions
0,2000-03-21,200001CP00079,"8:06am, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510986,...,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN,
1,2000-03-21,200001CP00131,"6:00pm, 21st March 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511196,...,4.4,13.7,NaN,NaN,86.0,10.2,NaN,1022.2,NaN,
2,2000-07-27,200001CP00264,"5:58pm, 27th July 2000",serious,Cyclist,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511908,...,16.0,23.8,NaN,NaN,228.0,10.0,NaN,1012.5,NaN,
3,2000-08-09,200001CP00309,"1:00pm, 9th August 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.510995,...,16.0,22.7,NaN,NaN,200.0,11.6,NaN,1018.6,NaN,
4,2000-11-20,200001CP00438,"6:15am, 20th November 2000",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511470,...,4.0,6.9,NaN,NaN,221.0,9.5,NaN,995.6,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,2019480951155,"6:30pm, 6th December 2019",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511124,...,7.1,10.8,1.5,NaN,245.0,25.2,46.3,1006.0,NaN,
134,2019-12-30,2019480932151,"3:28pm, 30th December 2019",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511032,...,3.7,9.5,0.0,NaN,223.0,11.7,25.9,1029.4,NaN,
135,2020-09-28,2020480990431,"12:10pm, 28th September 2020",slight,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.512006,...,8.4,15.8,0.0,NaN,299.0,13.7,29.6,1013.4,NaN,
136,2020-10-02,2020480992250,"7:16am, 2nd October 2020",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.511075,...,9.9,11.8,3.6,NaN,68.0,19.8,40.8,988.7,NaN,


In [103]:
#drop columns

df.drop(columns =['Accident_Index', 'Casualties', 'url', 'apiUrl', 'wdir', 'pres', 'tsun'], axis = 1)


,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,tavg,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Snow_depth_mm,Wind_speed_kmh,Peak_wind_gust_in_kmh,Weather_conditions
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,8.3,4.4,13.7,NaN,NaN,10.2,NaN,
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,8.3,4.4,13.7,NaN,NaN,10.2,NaN,
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,19.3,16.0,23.8,NaN,NaN,10.0,NaN,
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,19.1,16.0,22.7,NaN,NaN,11.6,NaN,
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,5.8,4.0,6.9,NaN,NaN,9.5,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,9.2,7.1,10.8,1.5,NaN,25.2,46.3,
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,5.8,3.7,9.5,0.0,NaN,11.7,25.9,
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,11.7,8.4,15.8,0.0,NaN,13.7,29.6,
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,11.1,9.9,11.8,3.6,NaN,19.8,40.8,


In [113]:
#Create a dictionary according info in my big dataset, I want everything to be consistent in both datasets

severity = {
            'slight':'Slight', 'serious':'Serious', 'fatal':'Fatal'}

In [114]:
#rename coulmn
df['Severity'] = df['Severity'].apply(lambda x: severity[x])

In [116]:
#drop-off columns

df.drop(columns =['Accident_Index', 'Casualties', 'url', 'apiUrl', 'wdir', 'pres', 'tsun'], axis = 1)

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,tavg,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Snow_depth_mm,Wind_speed_kmh,Peak_wind_gust_in_kmh,Weather_conditions
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,8.3,4.4,13.7,NaN,NaN,10.2,NaN,
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,8.3,4.4,13.7,NaN,NaN,10.2,NaN,
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,19.3,16.0,23.8,NaN,NaN,10.0,NaN,
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,19.1,16.0,22.7,NaN,NaN,11.6,NaN,
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,5.8,4.0,6.9,NaN,NaN,9.5,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,9.2,7.1,10.8,1.5,NaN,25.2,46.3,
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,5.8,3.7,9.5,0.0,NaN,11.7,25.9,
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,11.7,8.4,15.8,0.0,NaN,13.7,29.6,
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,11.1,9.9,11.8,3.6,NaN,19.8,40.8,


In [120]:
df.drop(columns =['Accident_Index', 'Casualties', 'url', 'apiUrl', 'wdir', 'pres', 'tsun'], axis = 1, inplace=True)

In [122]:
#save file to avoid runnig all cells

df.to_csv('/Users/kristina/Documents/Velo_python/allweather_data02.csv')

In [123]:
df = pd.read_csv('/Users/kristina/Documents/Velo_python/allweather_data02.csv')

In [125]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Date', 'Timestamp', 'Severity',
       'Number_of_Casualties', 'Number_of_Vehicles', 'Latitude', 'Longitude',
       'tavg', 'Min_temperature_C', 'Max_temperature_C',
       'Daily_precipitation_mm', 'Snow_depth_mm', 'Wind_speed_kmh',
       'Peak_wind_gust_in_kmh', 'Weather_conditions'],
      dtype='object')

In [126]:
df.drop(columns =['Unnamed: 0.1', 'Unnamed: 0'], axis = 1, inplace=True)

After examining the data, I see that the precipitation column contains some NaNs. 

I checked the information for those days additionally. 
It seems that on those dates the data was collected by the oldest weather stations in the city, one of which is right in central London.

I believe that if there really was precipitation on those days, the weather stations would have recorded it. 
So I assume in this case that the lack of precipitation data on these days means that there was no precipitation and then I write in the "Weather Conditions" column - "Clear"

In [128]:
#in percipation NaN - Clear

df.loc[df.Daily_precipitation_mm.isnull(), 'Weather_conditions']='Clear'

In [129]:
#if percipation less then 3 mm - Clear

df.loc[df.Daily_precipitation_mm <= 3.0, 'Weather_conditions']='Clear'

In [130]:
#if percipation more then 3 mm - Rain

df.loc[df.Daily_precipitation_mm > 3.0, 'Weather_conditions']='Rain'

In [132]:
df['Weather_conditions'].count()

138

In [133]:
df.loc[df.Daily_precipitation_mm > 0.0].count()

Date                      59
Timestamp                 59
Severity                  59
Number_of_Casualties      59
Number_of_Vehicles        59
Latitude                  59
Longitude                 59
tavg                      59
Min_temperature_C         59
Max_temperature_C         59
Daily_precipitation_mm    59
Snow_depth_mm              0
Wind_speed_kmh            59
Peak_wind_gust_in_kmh      3
Weather_conditions        59
dtype: int64

In [134]:
df['Snow_depth_mm'].unique()

array([nan, 20.])

In [135]:
df.loc[df.Snow_depth_mm >=20.0, 'Weather_conditions']='Snow'

In [142]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,tavg,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Snow_depth_mm,Wind_speed_kmh,Peak_wind_gust_in_kmh,Weather_conditions
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,8.3,4.4,13.7,NaN,NaN,10.2,NaN,Clear
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,8.3,4.4,13.7,NaN,NaN,10.2,NaN,Clear
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,19.3,16.0,23.8,NaN,NaN,10.0,NaN,Clear
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,19.1,16.0,22.7,NaN,NaN,11.6,NaN,Clear
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,5.8,4.0,6.9,NaN,NaN,9.5,NaN,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,9.2,7.1,10.8,1.5,NaN,25.2,46.3,Clear
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,5.8,3.7,9.5,0.0,NaN,11.7,25.9,Clear
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,11.7,8.4,15.8,0.0,NaN,13.7,29.6,Clear
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,11.1,9.9,11.8,3.6,NaN,19.8,40.8,Rain


To understand what I might consider a windy day, I use data from wikipedia
https://en.wikipedia.org/wiki/Beaufort_scale

I decide that it is the wind gusts that represent how windy the day was and use that column to estimate the wind.

In [144]:
df["Wind"] = ""

In [155]:
df.loc[df.Peak_wind_gust_in_kmh <=19 , 'Wind']=''

In [156]:
df.loc[df.Peak_wind_gust_in_kmh > 19.1 , 'Wind']= 'Windy'

In [157]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,tavg,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Snow_depth_mm,Wind_speed_kmh,Peak_wind_gust_in_kmh,Weather_conditions,Wind,Road_conditions
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,8.3,4.4,13.7,NaN,NaN,10.2,NaN,Clear,,Dry
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,8.3,4.4,13.7,NaN,NaN,10.2,NaN,Clear,,Dry
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,19.3,16.0,23.8,NaN,NaN,10.0,NaN,Clear,,Dry
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,19.1,16.0,22.7,NaN,NaN,11.6,NaN,Clear,,Dry
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,5.8,4.0,6.9,NaN,NaN,9.5,NaN,Clear,,Dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,9.2,7.1,10.8,1.5,NaN,25.2,46.3,Clear,Windy,Dry
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,5.8,3.7,9.5,0.0,NaN,11.7,25.9,Clear,Windy,Dry
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,11.7,8.4,15.8,0.0,NaN,13.7,29.6,Clear,Windy,Dry
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,11.1,9.9,11.8,3.6,NaN,19.8,40.8,Rain,Windy,Wet


I'm trying to fill road condition data based on the information I have.

On clear days I write "Dry", on rainy days I write "Wet", on days when the air temperature was near zero and below, I write "Frost"

Of course, these are my assumptions, but as a part of my learning, I want to fill in these values and see the visualization.

In [149]:
#'Snow', 'Wet', 'Dry', 'Frost', 'Flood'

df["Road_conditions"] = ""

In [150]:
df.loc[df.Weather_conditions == "Clear" , 'Road_conditions']= 'Dry'

In [151]:
df.loc[df.Weather_conditions == "Rain" , 'Road_conditions']= 'Wet'

In [152]:
df.loc[df.Weather_conditions == "Snow" , 'Road_conditions']= 'Snow'

In [153]:
df.loc[df.Min_temperature_C <=0 , 'Road_conditions']= 'Frost'

In [161]:
#joining two string columns with a space and save some more solutions as a comment
#[''.join(i) for i in zip(df["Weather_conditions"].map(str),df["Wind"])] <150
#df['Weather_conditions'].astype(str) + df['Wind'] > 150 rows

df['Weather_conditions_'] = df[['Weather_conditions', 'Wind']].agg(' '.join, axis=1)

In [162]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,tavg,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Snow_depth_mm,Wind_speed_kmh,Peak_wind_gust_in_kmh,Weather_conditions,Wind,Road_conditions,Weather_conditions_
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,8.3,4.4,13.7,NaN,NaN,10.2,NaN,Clear,,Dry,Clear
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,8.3,4.4,13.7,NaN,NaN,10.2,NaN,Clear,,Dry,Clear
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,19.3,16.0,23.8,NaN,NaN,10.0,NaN,Clear,,Dry,Clear
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,19.1,16.0,22.7,NaN,NaN,11.6,NaN,Clear,,Dry,Clear
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,5.8,4.0,6.9,NaN,NaN,9.5,NaN,Clear,,Dry,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,9.2,7.1,10.8,1.5,NaN,25.2,46.3,Clear,Windy,Dry,Clear Windy
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,5.8,3.7,9.5,0.0,NaN,11.7,25.9,Clear,Windy,Dry,Clear Windy
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,11.7,8.4,15.8,0.0,NaN,13.7,29.6,Clear,Windy,Dry,Clear Windy
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,11.1,9.9,11.8,3.6,NaN,19.8,40.8,Rain,Windy,Wet,Rain Windy


In [163]:
#save to csv to avoid running sells from above if I need to update something

df.to_csv('/Users/kristina/Documents/Velo_python/allweather_data03.csv')

In [164]:
df = pd.read_csv ('/Users/kristina/Documents/Velo_python/allweather_data03.csv')

In [169]:
df.columns

Index(['Unnamed: 0', 'Date', 'Timestamp', 'Severity', 'Number_of_Casualties',
       'Number_of_Vehicles', 'Latitude', 'Longitude', 'tavg',
       'Min_temperature_C', 'Max_temperature_C', 'Daily_precipitation_mm',
       'Snow_depth_mm', 'Wind_speed_kmh', 'Peak_wind_gust_in_kmh',
       'Weather_conditions', 'Wind', 'Road_conditions', 'Weather_conditions_'],
      dtype='object')

In [170]:


df.drop(columns =['Unnamed: 0', 'tavg', 'Snow_depth_mm', 'Wind_speed_kmh', 'Peak_wind_gust_in_kmh', 'Weather_conditions', 'Wind'], axis = 1, inplace=True)

In [171]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Road_conditions,Weather_conditions_
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,4.4,13.7,NaN,Dry,Clear
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,4.4,13.7,NaN,Dry,Clear
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,16.0,23.8,NaN,Dry,Clear
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,16.0,22.7,NaN,Dry,Clear
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,4.0,6.9,NaN,Dry,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,7.1,10.8,1.5,Dry,Clear Windy
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,3.7,9.5,0.0,Dry,Clear Windy
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,8.4,15.8,0.0,Dry,Clear Windy
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,9.9,11.8,3.6,Wet,Rain Windy


In [172]:
df['Month'] = pd.to_datetime(df['Date']).dt.month

In [177]:
df['Day'] = pd.to_datetime(df['Timestamp']).dt.day_name()

In [179]:
df['Time'] = pd.to_datetime(df['Timestamp']).dt.time

In [180]:
season = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}

In [181]:
df['Season'] = df['Month'].apply(lambda x: season[x])

In [188]:
Precipation_0_1 = {
            'Rain ':1, 'Snow ':1, 'Fog':1,
            'Clear ': 0, 'Clear Windy': 0, 'Rain Windy':1,
           'Snow Windy':1}

In [189]:
df['Precipation_0_1'] = df['Weather_conditions_'].apply(lambda x: Precipation_0_1[x])

In [192]:
df['Month'] = pd.to_datetime(df['Timestamp']).dt.month_name()

In [193]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Road_conditions,Weather_conditions_,Month,Day,Time,Season,Precipation_0_1
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,4.4,13.7,NaN,Dry,Clear,March,Tuesday,08:06:00,Spring,0
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,4.4,13.7,NaN,Dry,Clear,March,Tuesday,18:00:00,Spring,0
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,16.0,23.8,NaN,Dry,Clear,July,Thursday,17:58:00,Summer,0
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,16.0,22.7,NaN,Dry,Clear,August,Wednesday,13:00:00,Summer,0
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,4.0,6.9,NaN,Dry,Clear,November,Monday,06:15:00,Autumn,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,7.1,10.8,1.5,Dry,Clear Windy,December,Friday,18:30:00,Winter,0
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,3.7,9.5,0.0,Dry,Clear Windy,December,Monday,15:28:00,Winter,0
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,8.4,15.8,0.0,Dry,Clear Windy,September,Monday,12:10:00,Autumn,0
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,9.9,11.8,3.6,Wet,Rain Windy,October,Friday,07:16:00,Autumn,1


In [194]:
#save to csv to avoid running sells from above if I need to update something

df.to_csv('/Users/kristina/Documents/Velo_python/allweather_data04.csv')

In [207]:
df = pd.read_csv ('/Users/kristina/Documents/Velo_python/allweather_data04.csv')

In [208]:
workingdays = {
           'Monday':'working_day', 
           'Tuesday':'working_day', 
           'Wednesday':'working_day',
            'Thursday':'working_day', 
            'Friday':'pre_weekend', 
            'Saturday':'weekend',
            'Sunday':'weekend'}

In [209]:
df['Working_days'] = df['Day'].apply(lambda x: workingdays[x])

In [210]:
df

,Unnamed: 0,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,Min_temperature_C,Max_temperature_C,Daily_precipitation_mm,Road_conditions,Weather_conditions_,Month,Day,Time,Season,Precipation_0_1,Working_days
0,0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,4.4,13.7,NaN,Dry,Clear,March,Tuesday,08:06:00,Spring,0,working_day
1,1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,4.4,13.7,NaN,Dry,Clear,March,Tuesday,18:00:00,Spring,0,working_day
2,2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,16.0,23.8,NaN,Dry,Clear,July,Thursday,17:58:00,Summer,0,working_day
3,3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,16.0,22.7,NaN,Dry,Clear,August,Wednesday,13:00:00,Summer,0,working_day
4,4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,4.0,6.9,NaN,Dry,Clear,November,Monday,06:15:00,Autumn,0,working_day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,7.1,10.8,1.5,Dry,Clear Windy,December,Friday,18:30:00,Winter,0,pre_weekend
134,134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,3.7,9.5,0.0,Dry,Clear Windy,December,Monday,15:28:00,Winter,0,working_day
135,135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,8.4,15.8,0.0,Dry,Clear Windy,September,Monday,12:10:00,Autumn,0,working_day
136,136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,9.9,11.8,3.6,Wet,Rain Windy,October,Friday,07:16:00,Autumn,1,pre_weekend


For more granular data, 
I processed the data from the timestamp to get 
the names of the months, 
the names of the days, 
the names of the seasons, 
whether there was precipitation on this day or not, 
and whether it was a working day, weekend or Friday (I called it pre-weekend, because Friday has its own specifics in people's lives)

Now I want to remove unnecessary columns and save a file with data that I will need for visualization

In [212]:
df.columns

Index(['Unnamed: 0', 'Date', 'Timestamp', 'Severity', 'Number_of_Casualties',
       'Number_of_Vehicles', 'Latitude', 'Longitude', 'Min_temperature_C',
       'Max_temperature_C', 'Daily_precipitation_mm', 'Road_conditions',
       'Weather_conditions_', 'Month', 'Day', 'Time', 'Season',
       'Precipation_0_1', 'Working_days'],
      dtype='object')

In [213]:
df.drop(columns =['Unnamed: 0', 'Min_temperature_C', 'Max_temperature_C', 
                  'Daily_precipitation_mm'], axis = 1, inplace=True)

In [214]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,Road_conditions,Weather_conditions_,Month,Day,Time,Season,Precipation_0_1,Working_days
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,Dry,Clear,March,Tuesday,08:06:00,Spring,0,working_day
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,Dry,Clear,March,Tuesday,18:00:00,Spring,0,working_day
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,Dry,Clear,July,Thursday,17:58:00,Summer,0,working_day
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,Dry,Clear,August,Wednesday,13:00:00,Summer,0,working_day
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,Dry,Clear,November,Monday,06:15:00,Autumn,0,working_day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,Dry,Clear Windy,December,Friday,18:30:00,Winter,0,pre_weekend
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,Dry,Clear Windy,December,Monday,15:28:00,Winter,0,working_day
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,Dry,Clear Windy,September,Monday,12:10:00,Autumn,0,working_day
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,Wet,Rain Windy,October,Friday,07:16:00,Autumn,1,pre_weekend


In [215]:
#save to csv
df.to_csv('/Users/kristina/Documents/Velo_python/london_data_clean.csv', index=False)

In [216]:
df = pd.read_csv ('/Users/kristina/Documents/Velo_python/london_data_clean.csv')

In [217]:
df

,Date,Timestamp,Severity,Number_of_Casualties,Number_of_Vehicles,Latitude,Longitude,Road_conditions,Weather_conditions_,Month,Day,Time,Season,Precipation_0_1,Working_days
0,2000-03-21,"8:06am, 21st March 2000",Slight,1,2,51.510986,-0.102536,Dry,Clear,March,Tuesday,08:06:00,Spring,0,working_day
1,2000-03-21,"6:00pm, 21st March 2000",Slight,1,2,51.511196,-0.104401,Dry,Clear,March,Tuesday,18:00:00,Spring,0,working_day
2,2000-07-27,"5:58pm, 27th July 2000",Serious,1,1,51.511908,-0.103939,Dry,Clear,July,Thursday,17:58:00,Summer,0,working_day
3,2000-08-09,"1:00pm, 9th August 2000",Slight,1,2,51.510995,-0.103112,Dry,Clear,August,Wednesday,13:00:00,Summer,0,working_day
4,2000-11-20,"6:15am, 20th November 2000",Slight,1,2,51.511470,-0.104678,Dry,Clear,November,Monday,06:15:00,Autumn,0,working_day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2019-12-06,"6:30pm, 6th December 2019",Slight,1,2,51.511124,-0.107082,Dry,Clear Windy,December,Friday,18:30:00,Winter,0,pre_weekend
134,2019-12-30,"3:28pm, 30th December 2019",Serious,1,2,51.511032,-0.103036,Dry,Clear Windy,December,Monday,15:28:00,Winter,0,working_day
135,2020-09-28,"12:10pm, 28th September 2020",Slight,1,2,51.512006,-0.104307,Dry,Clear Windy,September,Monday,12:10:00,Autumn,0,working_day
136,2020-10-02,"7:16am, 2nd October 2020",Serious,1,2,51.511075,-0.102386,Wet,Rain Windy,October,Friday,07:16:00,Autumn,1,pre_weekend


Final check

In [220]:
df.isnull().count()

Date                    138
Timestamp               138
Severity                138
Number_of_Casualties    138
Number_of_Vehicles      138
Latitude                138
Longitude               138
Road_conditions         138
Weather_conditions_     138
Month                   138
Day                     138
Time                    138
Season                  138
Precipation_0_1         138
Working_days            138
dtype: int64